In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import math
import torch
from torch import nn
from d2l import torch as d2l

In [21]:
origin_train_data = pd.read_csv('./data/house/kaggle_house_real_train.csv')[:10000]
origin_test_data = pd.read_csv('./data/house/kaggle_house_real_test.csv')[:10000]

In [22]:
origin_train_data.shape, origin_test_data.shape

((10000, 41), (10000, 40))

### 清洗数据

去除一些干扰列，并合并数据集以便进一步清洗

In [23]:
cleaned_train_data = origin_train_data.drop(["Id", "Sold Price", "State"], axis=1)
cleaned_test_data = origin_test_data.drop(["Id", "State"], axis=1)

all_features = pd.concat((cleaned_train_data, cleaned_test_data))
all_features.shape

(20000, 38)

单位化数据

In [5]:
def numeric_features_unitization(data):
    numeric_features = data.select_dtypes(include=np.number).columns
    """ 以下式子等价于 data[numeric_features] = (data[numeric_features] - data[numeric_features].mean()) / data[numeric_features].std()"""
    data[numeric_features] = data[numeric_features].apply(lambda x : (x - x.mean()) / x.std())
    data[numeric_features] = data[numeric_features].fillna(0)
    return data

`one-hot`处理

In [12]:
def nonnumeric_features_dummies(data):
    return pd.get_dummies(data, dummy_na=True)

In [24]:
all_features = numeric_features_unitization(all_features)
all_features = nonnumeric_features_dummies(all_features)

In [27]:
def transform_numpy_to_torch(combination_data):
    num_sample_train = origin_train_data.shape[0]
    train_features = torch.tensor(combination_data[:num_sample_train].values.astype(float), dtype=torch.float32)
    test_features = torch.tensor(combination_data[num_sample_train:].values.astype(float), dtype=torch.float32)
    train_labels = torch.tensor(origin_train_data["Sold Price"].values.astype(float).reshape(-1, 1), dtype=torch.float32)
    return train_features, test_features, train_labels

In [28]:
train_features, test_features, train_labels = transform_numpy_to_torch(all_features)

MemoryError: Unable to allocate 4.90 GiB for an array with shape (65803, 10000) and data type object